# Recurrent Neural Network

In [ ]:
# Prepare a time series model for prediction of next day temperature
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


In [ ]:
# laod data set
df = pd.read_csv("daily_minimum_temps.csv",parse_dates=["Date"] , index_col = "Date")

/tmp/ipython-input-2-600307691.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv("daily_minimum_temps.csv",parse_dates=["Date"] , index_col = "Date")


In [ ]:
#Checking the first 5 rows
df.head()

,Temp
Date,
1981-01-01,20.7
1981-01-02,17.9
1981-01-03,18.8
1981-01-04,14.6
1981-01-05,15.8


In [ ]:
# finding out the missing values
df.isna().sum(axis=0)

,0
Temp,0


In [ ]:
df.shape

(3650, 1)

In [ ]:
# converting the string values to numeric values by removig the double quotes and if string value does not make any sense or it
# We convert that value to null.
df["Temp"] = pd.to_numeric(df["Temp"], errors="coerce")

In [ ]:
df=df.dropna()

In [ ]:
# Normalize the Features
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df["Temp"].values.reshape(-1,1))

In [ ]:
# Sequence lenght for temperature
seq_len = 30

In [ ]:
# Function for creating sequences
def create_sequences(data_scaled, seq_len):
  X,y = [],[]
  for i in range(len(data_scaled)-seq_len):
    X.append(data_scaled[i:i+seq_len])
    y.append(data_scaled[i+seq_len])
  return np.array(X),np.array(y)

In [ ]:
#calling the fucntion and storng the values in list X and y
X,y=create_sequences(data_scaled, seq_len)

In [ ]:
# divide the datset into train test and split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,shuffle=False,random_state=1)

In [ ]:
# bulidiing the Rnn model
model = Sequential([
    LSTM(64, activation="relu", input_shape=(seq_len,1)),
    Dense(1) # output will be a single value
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# Complie The model
model.compile(optimizer="adam", loss="mse")

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32)

Epoch 1/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0670
Epoch 2/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0111
Epoch 3/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0103
Epoch 4/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0109
Epoch 5/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0103
Epoch 6/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0106
Epoch 7/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0102
Epoch 8/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0107
Epoch 9/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0106
Epoch 10/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0100
Epoch 11/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.0097
Epoch 12/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0096
Epoch 13/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0092
Epoch 14/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0093
Epoch 15/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0095
Epoch 16/20

In [ ]:
# make Predicions
y_pred_scaled = model.predict(X_test)

In [ ]:
# inversew tranfrom the scaled data
y_pred_scaled = np.clip(y_pred_scaled, 0, 1)
y_pred = scaler.inverse_transform(y_pred_scaled)
t_test_actual = scaler.inverse_transform(y_test)

In [ ]:
# Predict the next day temperature
last_sequence = data_scaled[-seq_len:].reshape(1,seq_len,1)
next_temp_scaled = model.predict(last_sequence)
next_temp_scaled = np.clip(next_temp_scaled, 0, 1)
next_day_temp = scaler.inverse_transform(next_temp_scaled)
print("Next day temperature prediction:", next_day_temp)

In [ ]:
import joblib
joblib.dump(model, 'temperature_prediction_model.pkl')
joblib.dump(scaler, 'temperature_scaler.pkl')